In [1]:
import numpy as np
import pandas as pd

from sklearn.ensemble import ExtraTreesRegressor

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
from IPython.display import clear_output

In [2]:
train = pd.read_csv(".././data/train.csv", parse_dates=['epoch'])
test = pd.read_csv(".././data/test.csv", parse_dates=['epoch'])

In [3]:
def smape(y_pred, y_true): 
    return 'SMAPE', np.mean(np.abs((y_pred - y_true)/(np.abs(y_pred) + np.abs(y_true)))), False

In [6]:
def create_lag_features(df, steps):
    sim_cols = ['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']
    for i in range(df['sat_id'].nunique()):
        idx = df['sat_id'].unique()[i]
        clear_output()
        print("Now processing {} of {}".format(i, df['sat_id'].nunique()))
        data = df[df['sat_id'] == idx]
        for col in sim_cols:
            for step in range(1, steps+1):
                b_roll = np.roll(data[col].values, step)
                b_roll[:step] = b_roll[step]
                df.loc[df['sat_id']==idx, col+'_b'+str(step)] = b_roll
                
                f_roll = np.roll(data[col].values, -step)
                f_roll[-step:] = f_roll[-step]
                df.loc[df['sat_id']==idx, col+'_f'+str(step)] = f_roll
    print("[INFO] Lag features ready!")
    return df

In [11]:
train = create_lag_features(train, steps=2)
test = create_lag_features(test, steps=2)

Now processing 299 of 300
[INFO] Lag features ready!


In [12]:
train.to_csv('.././mod_data/train_lag2.csv', index=False)
test.to_csv('.././mod_data/test_lag2.csv', index=False)

In [14]:
train_cols = ['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']
lag_cols = [col for col in train.columns if ('_b' in col) or ('_f' in col)]
reg = ExtraTreesRegressor(n_estimators=500, random_state=42)

def fill_missing_lag(df, train_cols, lag_cols):
    for i in range(df['sat_id'].nunique()):
        idx = df['sat_id'].unique()[i]
        clear_output()
        print("Now processing {} of {}".format(i, df['sat_id'].nunique()))
        data = df[df['sat_id'] == idx]
        for col in lag_cols:
            X_train = data.loc[data[col].notnull(), train_cols].values
            y_train = data.loc[data[col].notnull(), col].values
            X_test = data.loc[data[col].isnull(), train_cols].values
            reg.fit(X_train, y_train)
            df.loc[(df['sat_id']==idx) & (df[col].isnull()), col] = reg.predict(X_test)
    return df

In [15]:
train = fill_missing_lag(train, train_cols, lag_cols)
test = fill_missing_lag(test, train_cols, lag_cols)

Now processing 0 of 600


ValueError: Found array with 0 sample(s) (shape=(0, 6)) while a minimum of 1 is required.